## AURA

#### Imports

In [1]:
!pip install evaluate
!pip install bert_score
!pip install rouge-score
!pip install meteor
!pip install nltk

In [2]:
import argparse
import evaluate
import json
import math
import nltk
import numpy as np
import pandas as pd
import os
import random
import re
import torch
import torch.nn as nn
import torch.nn.functional as F

from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import EnglishStemmer
from nltk.tokenize import sent_tokenize
from sklearn.metrics import mean_squared_error, mean_absolute_error, precision_score, recall_score, f1_score
from sklearn.preprocessing import Binarizer
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
from transformers import T5ForConditionalGeneration, T5Tokenizer
from typing import Any, Dict, List, Tuple, Union

In [3]:
nltk.download("punkt_tab")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

### Model

In [4]:
def attention_function(Q: torch.Tensor, K: torch.Tensor, V: torch.Tensor,
                       mask: torch.Tensor=None) -> Tuple[torch.Tensor]:
    attention_scores = F.softmax(
        torch.matmul(Q, K.transpose(1, 2)) / torch.sqrt(torch.tensor(Q.size(-1))),
        dim=-1
    )
    if mask is not None:
        mask = mask.unsqueeze(1) if mask.dim() == 2 else mask
        attention_scores = attention_scores.masked_fill(mask == 0, -1e9)
    attention_outputs = torch.matmul(attention_scores, V)
    return attention_outputs, attention_scores


class Attention(nn.Module):

    def __init__(self, d):
        super().__init__()
        self.d = d
        self.WQ = nn.Linear(d, d)
        self.WK = nn.Linear(d, d)
        self.WV = nn.Linear(d, d)

    def forward(self, Q: torch.Tensor, K: torch.Tensor, V: torch.Tensor,
                mask: torch.Tensor=None) -> Tuple[torch.Tensor]:
        Q = self.WQ(Q)
        K = self.WK(K)
        V = self.WV(V)

        attention_outputs, attention_scores = attention_function(Q, K, V, mask)
        return attention_outputs, attention_scores


class TextModule:

    def encode(self, tokens: torch.Tensor, mask: torch.Tensor=None) -> torch.Tensor:
        # tokens: (batch_size, seq_len)
        # mask: (batch_size, seq_len)
        raise NotImplementedError

    def decode(self, embeddings: torch.Tensor, labels: torch.Tensor=None) -> torch.Tensor:
        # embeddings: (batch_size, seq_len, d_words)
        # labels: (batch_size, seq_len)
        raise NotImplementedError

    def generate(self, embeddings: torch.Tensor) -> List[str]:
        # embeddings: (batch_size, seq_len, d_words)
        raise NotImplementedError


class T5TextModule(nn.Module, TextModule):

    def __init__(self, config, t5_model):
        super().__init__()
        self.config = config
        self.t5_model = t5_model

    def forward(self,
                tokens: torch.Tensor=None, mask: torch.Tensor=None,
                embeddings: torch.Tensor=None, labels: torch.Tensor=None,
                encode: bool=False, decode: bool=False) -> torch.Tensor:
        if encode and tokens is not None:
            return self.t5_model.encoder(input_ids=tokens, attention_mask=mask).last_hidden_state
        if decode and embeddings is not None:
            return self.t5_model(inputs_embeds=embeddings, labels=labels).loss

    def encode(self, tokens: torch.Tensor, mask: torch.Tensor=None) -> torch.Tensor:
        return self.forward(tokens=tokens, mask=mask, encode=True)

    def decode(self, embeddings: torch.Tensor, labels: torch.Tensor=None) -> torch.Tensor:
        return self.forward(embeddings=embeddings, labels=labels, decode=True)

    def generate(self, embeddings: torch.Tensor) -> torch.Tensor:
        return self.t5_model.generate(
            inputs_embeds=embeddings,
            do_sample=False,
            max_length=self.config.review_length,
            top_k=50,
            top_p=0.95,
            repetition_penalty=1.2
        )


class RatingsLoss(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.config = config
        self.overall_rating_loss = nn.MSELoss()
        self.aspect_rating_loss = nn.MSELoss()
        self.review_rating_loss = nn.MSELoss()

    def forward(self, R: torch.Tensor, R_hat: torch.Tensor,
                A_ratings: torch.Tensor, A_ratings_hat: torch.Tensor,
                reviews_ratings: torch.Tensor, reviews_ratings_hat: torch.Tensor) -> Dict[str, torch.Tensor]:

        overall_rating_loss = self.overall_rating_loss(R_hat, R)
        aspect_rating_loss = self.aspect_rating_loss(A_ratings_hat.flatten(), A_ratings.flatten())
        review_rating_loss = self.review_rating_loss(reviews_ratings_hat.flatten(), reviews_ratings.flatten())

        total_loss = (
            self.config.alpha * overall_rating_loss +
            self.config.beta * aspect_rating_loss +
            self.config.gamma * review_rating_loss
        )

        return {
            "total": total_loss,
            "overall_rating": overall_rating_loss,
            "aspect_rating": aspect_rating_loss,
            "review_rating": review_rating_loss
        }


class SentenceEmbedding(nn.Module):

    def __init__(self, config, encoder=None):
        super().__init__()
        self.config = config
        self.encoder = encoder
        #self.sentence_attention = Attention(config.d_words)

    def encode(self, sentence_tokens: torch.Tensor, mask: torch.Tensor) -> torch.Tensor:
        return self.encoder.encode(sentence_tokens, mask)

    def forward(self, E_embeddings: torch.Tensor,
                sentence_tokens: torch.Tensor, mask: torch.Tensor=None) -> Dict[str, torch.Tensor]:
        # E_embeddings: (batch_size, d_words)
        # sentence_tokens, mask: (batch_size, seq_len)

        tokens_embeddings = self.encode(sentence_tokens, mask) # (batch_size, seq_len, d_words)
        sentence_embeddings, attention_scores = attention_function(
            Q=E_embeddings.unsqueeze(1), K=tokens_embeddings, V=tokens_embeddings, mask=mask
        )
        attention_scores = attention_scores.squeeze(1) # (batch_size, seq_len)
        sentence_embeddings = sentence_embeddings.squeeze(1) # (batch_size, d_words)

        _out = {
            "embeddings": sentence_embeddings,
            "attention": attention_scores
        }
        return _out


class DocumentEmbedding(nn.Module):

    def __init__(self, config, encoder=None):
        super().__init__()
        self.config = config
        #self.document_attention = Attention(config.d_words)
        self.sentence_embedding = SentenceEmbedding(config, encoder)
        self.overall_rating = nn.Sequential(
            nn.Linear(self.config.d_words, self.config.d_model),
            nn.ReLU(),
            nn.Linear(self.config.d_model, 1)
        )

    def forward(self, E_embeddings: torch.Tensor,
                document_tokens: torch.Tensor, mask: torch.Tensor=None) -> Dict[str, torch.Tensor]:
        # E_embeddings: (batch_size, d_words)
        # document_tokens, attention_mask: (batch_size, n_sentences, seq_len)

        batch_size, n_sentences, seq_len = document_tokens.size()

        document_tokens_flat = document_tokens.view(-1, seq_len) # (batch_size * n_sentences, seq_len)
        mask_flat = mask.view(-1, seq_len) if mask is not None else None
        E_embeddings_flat = E_embeddings.repeat(n_sentences, 1) # (batch_size * n_sentences, d_words)

        sentence_outputs = self.sentence_embedding(E_embeddings_flat, document_tokens_flat, mask_flat)
        sentences_embeddings = sentence_outputs["embeddings"].view(batch_size, n_sentences, -1) # (batch_size, n_sentences, d_words)

        document_mask = mask.sum(dim=-1) # (batch_size, n_sentences)
        document_embeddings, attention_scores = attention_function(
            Q=E_embeddings.unsqueeze(1) , K=sentences_embeddings, V=sentences_embeddings, mask=document_mask
        )
        attention_scores = attention_scores.squeeze(1) # (batch_size, n_sentences)
        document_embeddings = document_embeddings.squeeze(1) # (batch_size, d_words)

        rating = self.overall_rating(document_embeddings).squeeze(1) # (batch_size,)
        rating = torch.clamp(rating, min=self.config.min_rating, max=self.config.max_rating)

        _out = {
            "embeddings": document_embeddings,
            "attention": attention_scores,
            "rating": rating,
            "sentence" : {
                "embeddings": sentences_embeddings,
                "attention": sentence_outputs["attention"].view(batch_size, n_sentences, seq_len)
            }
        }
        return _out


class AURA(nn.Module):

    def __init__(self, config, text_module, tokenizer):
        super().__init__()
        self.config = config
        self.text_module = text_module
        self.tokenizer = tokenizer
        self.ratings_loss = RatingsLoss(config)

        self.user_embedding = nn.Embedding(config.n_users, config.d_model)
        self.item_embedding = nn.Embedding(config.n_items, config.d_model)

        self.user_document_embedding = DocumentEmbedding(config, text_module)
        self.item_document_embedding = DocumentEmbedding(config, text_module)

        self.user_words_proj = nn.Linear(config.d_model, config.d_words)
        self.item_words_proj = nn.Linear(config.d_model, config.d_words)

        self.user_aspects_embedding = nn.ModuleList([
            nn.Sequential(
                nn.Linear(config.d_model, config.d_model),
                nn.ReLU(),
                nn.Linear(config.d_model, config.d_model)
            ) for _ in range(config.n_aspects)
        ])
        self.item_aspects_embedding = nn.ModuleList([
            nn.Sequential(
                nn.Linear(config.d_model, config.d_model),
                nn.ReLU(),
                nn.Linear(config.d_model, config.d_model)
            ) for _ in range(config.n_aspects)
        ])

        self.aspects_rating = nn.ModuleList([
            nn.Sequential(
                nn.Linear(2 * config.d_model, config.d_model),
                nn.ReLU(),
                nn.Linear(config.d_model, 1)
            ) for _ in range(config.n_aspects)
        ])

        self.overall_rating = nn.Sequential(
            nn.Linear(2 * config.d_model, config.d_model),
            nn.ReLU(),
            nn.Linear(config.d_model, 1)
        )

        self.prompt_embedding = nn.Sequential(
            nn.Linear(2 * config.d_words, config.n_prompt * config.d_words),
            nn.ReLU(),
            nn.Linear(config.n_prompt * config.d_words, config.n_prompt * config.d_words)
        )

        self.training_phase = 0 # 0: ratings, 1: prompt, 2: text module

    def set_training_phase(self, phase: int):
        self.training_phase = phase
        if phase == 0:
            self.ratings_grad(True)
            self.prompt_grad(False)
            self.text_grad(False)
        elif phase == 1:
            self.prompt_grad(True)
            self.ratings_grad(False)
            self.text_grad(False)
        #elif phase == 2:
        #    self.text_grad()

    def ratings_grad(self, flag: bool=True):
        for param in self.user_embedding.parameters():
            param.requires_grad = flag
        for param in self.item_embedding.parameters():
            param.requires_grad = flag
        for param in self.user_document_embedding.parameters():
            param.requires_grad = flag
        for param in self.item_document_embedding.parameters():
            param.requires_grad = flag
        for param in self.user_words_proj.parameters():
            param.requires_grad = flag
        for param in self.item_words_proj.parameters():
            param.requires_grad = flag
        for param in self.user_aspects_embedding.parameters():
            param.requires_grad = flag
        for param in self.item_aspects_embedding.parameters():
            param.requires_grad = flag
        for param in self.aspects_rating.parameters():
            param.requires_grad = flag
        for param in self.overall_rating.parameters():
            param.requires_grad = flag

    def prompt_grad(self, flag: bool=True):
        for param in self.prompt_embedding.parameters():
            param.requires_grad = flag

    def text_grad(self, flag: bool=True):
        for param in self.text_module.parameters():
            param.requires_grad = flag

    def forward(self, U_ids: torch.Tensor, I_ids: torch.Tensor,
                U_document_tokens: torch.Tensor=None, U_mask: torch.Tensor=None, U_ratings: torch.Tensor=None,
                I_document_tokens: torch.Tensor=None, I_mask: torch.Tensor=None, I_ratings: torch.Tensor=None,
                R: torch.Tensor=None, A_ratings: torch.Tensor=None,
                UI_review_tokens: torch.Tensor=None,
                inference_flag: bool=False) -> Dict[str, torch.Tensor]:
        # U_ids, I_ids: (batch_size,)
        # U_document_tokens, I_documents, U_mask, I_mask: (batch_size, n_sentences, seq_len)
        # UI_review_tokens: (batch_size, seq_len)

        batch_size = U_ids.size(0)

        U_embeddings = self.user_embedding(U_ids) # (batch_size, d_model)
        I_embeddings = self.item_embedding(I_ids) # (batch_size, d_model)

        U_embeddings_words = self.user_words_proj(U_embeddings) # (batch_size, d_words)
        I_embeddings_words = self.item_words_proj(I_embeddings) # (batch_size, d_words)

        U_outputs = self.user_document_embedding(U_embeddings_words, U_document_tokens, U_mask)
        I_outputs = self.item_document_embedding(I_embeddings_words, I_document_tokens, I_mask)

        _out = {}

        if inference_flag or self.training_phase == 0:
            UA_embeddings = []
            IA_embeddings = []
            A_ratings_hat = []
            for i in range(self.config.n_aspects):
                au_embedding = self.user_aspects_embedding[i](U_embeddings) # (batch_size, d_model)
                ai_embedding = self.item_aspects_embedding[i](I_embeddings) # (batch_size, d_model)
                a_rating = self.aspects_rating[i](torch.cat([au_embedding, ai_embedding], dim=-1)) # (batch_size,)
                a_rating = torch.clamp(a_rating, min=self.config.min_rating, max=self.config.max_rating)

                UA_embeddings.append(au_embedding)
                IA_embeddings.append(ai_embedding)
                A_ratings_hat.append(a_rating)

            UA_embeddings = torch.stack(UA_embeddings, dim=1) # (batch_size, n_aspects, d_model)
            IA_embeddings = torch.stack(IA_embeddings, dim=1) # (batch_size, n_aspects, d_model)
            A_ratings_hat = torch.stack(A_ratings_hat, dim=1).squeeze(2) # (batch_size, n_aspects)

            U_embeddings_aggregated, U_attention_scores = attention_function(
                Q=U_embeddings.unsqueeze(1), K=IA_embeddings, V=UA_embeddings
            )
            U_attention_scores = U_attention_scores.squeeze(1) # (batch_size, n_aspects)
            U_embeddings_aggregated = U_embeddings_aggregated.squeeze(1) # (batch_size, d_model)
            U_embeddings = U_embeddings + U_embeddings_aggregated

            I_embeddings_aggregated, I_attention_scores = attention_function(
                Q=I_embeddings.unsqueeze(1), K=UA_embeddings, V=IA_embeddings
            )
            I_attention_scores = I_attention_scores.squeeze(1) # (batch_size, n_aspects)
            I_embeddings_aggregated = I_embeddings_aggregated.squeeze(1) # (batch_size, d_model)
            I_embeddings = I_embeddings + I_embeddings_aggregated

            R_hat = self.overall_rating(torch.cat([U_embeddings, I_embeddings], dim=-1)).squeeze(1)
            R_hat = torch.clamp(R_hat, min=self.config.min_rating, max=self.config.max_rating)

            _out.update({
                "overall_rating": R_hat,
                "aspects_ratings": A_ratings_hat,
                "user": {
                    "attention": U_attention_scores,
                    "document": U_outputs
                },
                "item": {
                    "attention": I_attention_scores,
                    "document": I_outputs
                }
            })

            U_ratings_hat = U_outputs["rating"]
            I_ratings_hat = I_outputs["rating"]
            reviews_ratings_hat = torch.cat([U_ratings_hat, I_ratings_hat], dim=-1)
            reviews_ratings = torch.cat([U_ratings, I_ratings], dim=-1)

            losses = self.ratings_loss(R, R_hat, A_ratings, A_ratings_hat, reviews_ratings, reviews_ratings_hat)
            _out.update({"losses": losses})

        if inference_flag or self.training_phase == 1:
            P_embeddings = torch.cat([U_embeddings_words, I_embeddings_words], dim=1)
            P_embeddings = self.prompt_embedding(P_embeddings).view(batch_size, self.config.n_prompt, self.config.d_words)

            if not inference_flag:
                review_loss = self.text_module.decode(P_embeddings, UI_review_tokens)
                losses = {"review": review_loss, "total": review_loss}
                _out.update({"losses": losses})

            else:
                UI_review_tokens_hat = self.text_module.generate(P_embeddings)
                review = self.tokenizer.batch_decode(UI_review_tokens_hat, skip_special_tokens=True)
                _out.update({'review': review})

        return _out


### Data

In [5]:
class RatingsReviewDataset(Dataset):

        def __init__(self, data_df: pd.DataFrame, config: Any, tokenizer: T5Tokenizer):
            super().__init__()
            self.data_df = data_df
            self.tokenizer = tokenizer
            self.config = config

            self.users = self.data_df["user_id"].unique().tolist()
            self.items = self.data_df["item_id"].unique().tolist()

            self.users_index = {user_id: [] for user_id in self.users}
            self.items_index = {item_id: [] for item_id in self.items}

            self.ratings = self.data_df["rating"].tolist()
            self.reviews = self.data_df["review"].tolist()

            self.reviews_tokens = []
            self.reviews_masks = []

            self.sentences = []
            self.sentences_tokens = []
            self.sentences_masks = []
            self._sentence_tokens_pad = torch.full((self.config.sentence_length,),
                                                   self.tokenizer.pad_token_id, dtype=torch.long)
            self._sentence_masks_pad = torch.zeros((self.config.sentence_length,), dtype=torch.long)

            self._process()

        def __len__(self) -> int:
            return len(self.data_df)

        def _process(self):
            for index in tqdm(range(len(self)), desc="Processing data", colour="green"):
                row = self.data_df.iloc[index]

                user_id = row["user_id"]
                item_id = row["item_id"]

                self.users_index[user_id].append(index)
                self.items_index[item_id].append(index)

                review = self.reviews[index]
                review = preprocess_text(review, self.config, self.config.review_length)
                self.reviews[index] = review

                inputs = self.tokenizer(
                    review,
                    max_length=self.config.review_length,
                    truncation=True,
                    padding="max_length",
                    return_tensors="pt"
                )
                tokens = inputs["input_ids"].squeeze(0)
                masks = inputs["attention_mask"].squeeze(0)
                self.reviews_tokens.append(tokens)
                self.reviews_masks.append(masks)

                sentences = sent_tokenize(review)
                sentences = sentences[:min(len(sentences), self.config.n_sentences)]
                self.sentences.append(sentences)

                sentences_tokens = []
                sentences_masks = []
                for sentence in sentences:
                    inputs = self.tokenizer(
                        sentence,
                        max_length=self.config.sentence_length,
                        truncation=True,
                        padding="max_length",
                        return_tensors="pt"
                    )
                    tokens = inputs["input_ids"].squeeze(0)
                    masks = inputs["attention_mask"].squeeze(0)
                    sentences_tokens.append(tokens)
                    sentences_masks.append(masks)
                self.sentences_tokens.append(sentences_tokens)
                self.sentences_masks.append(sentences_masks)

        def __getitem__(self, index) -> Any:
            random.seed(index)
            row = self.data_df.iloc[index]

            user_id = row["user_id"]
            item_id = row["item_id"]

            overall_rating = row["rating"]
            aspects_ratings = [row[aspect] for aspect in self.config.aspects]

            review = self.reviews[index]
            review_tokens = self.reviews_tokens[index].clone()
            review_tokens[review_tokens == self.tokenizer.pad_token_id] = -100

            user_reviews_index = list(self.users_index[user_id])
            #user_reviews_index.remove(index)
            user_review_index = random.choice(user_reviews_index)
            user_document = self.sentences[user_review_index]
            user_document = user_document[:min(len(user_document), config.n_sentences)]
            user_document_tokens = self.sentences_tokens[user_review_index][:min(len(user_document), config.n_sentences)]
            user_document_masks = self.sentences_masks[user_review_index][:min(len(user_document), config.n_sentences)]
            if len(user_document_tokens) < self.config.n_sentences:
                pad_length = self.config.n_sentences - len(user_document)
                user_document_tokens.extend([self._sentence_tokens_pad.clone() for _ in range(pad_length)])
                user_document_masks.extend([self._sentence_masks_pad.clone() for _ in range(pad_length)])
            user_rating = self.ratings[user_review_index]

            item_reviews_index = list(self.items_index[item_id])
            #item_reviews_index.remove(index)
            item_review_index = random.choice(item_reviews_index)
            item_document = self.sentences[item_review_index]
            item_document = item_document[:min(len(item_document), config.n_sentences)]
            item_document_tokens = self.sentences_tokens[item_review_index][:min(len(item_document), config.n_sentences)]
            item_document_masks = self.sentences_masks[item_review_index][:min(len(item_document), config.n_sentences)]
            if len(item_document) < self.config.n_sentences:
                pad_length = self.config.n_sentences - len(item_document)
                item_document_tokens.extend([self._sentence_tokens_pad.clone() for _ in range(pad_length)])
                item_document_masks.extend([self._sentence_masks_pad.clone() for _ in range(pad_length)])
            item_rating = self.ratings[item_review_index]

            assert len(aspects_ratings) == self.config.n_aspects, str(len(aspects_ratings))
            assert len(user_document) <= self.config.n_sentences, str(len(user_document))
            assert len(item_document) <= self.config.n_sentences, str(len(item_document))
            assert len(user_document_tokens) == self.config.n_sentences, str(len(user_document_tokens))
            assert len(item_document_tokens) == self.config.n_sentences, str(len(item_document_tokens))
            assert len(user_document_masks) == self.config.n_sentences, str(len(user_document_masks))
            assert len(item_document_masks) == self.config.n_sentences, str(len(item_document_masks))

            random.seed(self.config.seed)

            return {
                "user_id": user_id,
                "item_id": item_id,
                "user_document_tokens": user_document_tokens,
                "item_document_tokens": item_document_tokens,
                "user_document_masks": user_document_masks,
                "item_document_masks": item_document_masks,
                "user_rating": user_rating,
                "item_rating": item_rating,
                "overall_rating": overall_rating,
                "aspects_ratings": aspects_ratings,
                "review": review,
                "review_tokens": review_tokens
            }


def collate_fn(batch):
    collated_batch = {}
    for key in batch[0]:
        values = [d[key] for d in batch]
        if isinstance(values[0], list) and isinstance(values[0][0], torch.Tensor):
            collated_batch[key] = torch.stack([torch.stack(tensor_list, dim=0) for tensor_list in values], dim=0)
        elif isinstance(values[0], torch.Tensor):
            collated_batch[key] = torch.stack(values, dim=0)
        else:
            collated_batch[key] = values
    return collated_batch


def process_data(data_df, config: Any) -> Tuple[pd.DataFrame]:
    data_df['user_id'] = data_df['user_id'].apply(str)
    data_df['item_id'] = data_df['item_id'].apply(str)

    users_vocab = create_vocab_from_df(data_df, 'user_id')
    items_vocab = create_vocab_from_df(data_df, 'item_id')

    data_df['user_id'] = data_df['user_id'].apply(lambda u: to_vocab_id(u, users_vocab))
    data_df['item_id'] = data_df['item_id'].apply(lambda i: to_vocab_id(i, items_vocab))

    train_df = data_df.sample(frac=config.train_size, random_state=config.seed)
    test_eval_df = data_df.drop(train_df.index)
    eval_size = config.eval_size / (config.eval_size + config.test_size)
    eval_df = test_eval_df.sample(frac=eval_size, random_state=config.seed)
    test_df = test_eval_df.drop(eval_df.index)

    return (train_df, eval_df, test_df), (users_vocab, items_vocab)

### Evalutaion

In [6]:
def rating_evaluation_pytorch(config: Any,
                              predictions: List[float], references: List[float],
                              users: List[float]=None) -> Dict[str, float]:

    results = {}

    actual_ratings = torch.tensor(references, dtype=torch.float32).to(config.device)
    predictions_tensor = torch.tensor(predictions, dtype=torch.float32).to(config.device)

    rmse = torch.sqrt(F.mse_loss(predictions_tensor, actual_ratings))
    mae = F.l1_loss(predictions_tensor, actual_ratings)

    results.update({'rmse': rmse.item(), 'mae': mae.item()})

    threshold = torch.tensor(config.threshold_rating).to(config.device)
    actual_binary = (actual_ratings >= threshold).float()
    predicted_binary = (predictions_tensor >= threshold).float()

    true_positives = (predicted_binary * actual_binary).sum()
    precision = true_positives / predicted_binary.sum() if predicted_binary.sum() > 0 else torch.tensor(1.0).to(config.device)
    recall = true_positives / actual_binary.sum() if actual_binary.sum() > 0 else torch.tensor(1.0).to(config.device)
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else torch.tensor(0.0).to(config.device)

    results.update({'precision': precision.item(), 'recall': recall.item(), 'f1': f1.item()})

    if not config.ranking_metrics_flag or users is None:
        return results

    users = list(set(users))

    ndcg_scores = []
    average_precisions = []
    #reciprocal_ranks = []

    for user in users:
        user_predictions = [pred for idx, pred in enumerate(predictions) if users[idx] == user]
        user_references = [ref for idx, ref in enumerate(references) if users[idx] == user]

        user_predictions_tensor = torch.tensor(user_predictions, dtype=torch.float32).to(config.device)

        sorted_indices = torch.configort(user_predictions_tensor, descending=True)
        relevances = torch.tensor(user_references, dtype=torch.float32).to(config.device)[sorted_indices]

        ndcg = ndcg_at_k_pytorch(relevances, config.k, config.device)
        ndcg_scores.append(ndcg.item())

        ap = average_precision_pytorch(relevances, config.threshold_rating, config.device)
        average_precisions.append(ap.item())

        #for rank, relevance in enumerate(relevances, start=1):
        #    if relevance >= config.threshold_rating:
        #        reciprocal_ranks.append(1 / rank)
        #        break
        #else:
        #   reciprocal_ranks.append(0)

    ndcg = torch.tensor(ndcg_scores).mean().item()
    map = torch.tensor(average_precisions).mean().item()
    #mrr = torch.tensor(reciprocal_ranks).mean().item()

    results.update({'ndcg': ndcg, 'map': map})

    return results


def rating_evaluation(config: Any,
                      predictions: List[float], references: List[float],
                      users: List[float]=None) -> Dict[str, float]:

    results = {}

    actual_ratings = references

    rmse = np.sqrt(mean_squared_error(actual_ratings, predictions))
    mae = mean_absolute_error(actual_ratings, predictions)

    results.update({'rmse': rmse, 'mae': mae})

    binarizer = Binarizer(threshold=config.threshold_rating)
    actual_binary = binarizer.fit_transform(actual_ratings.reshape(-1, 1)).flatten()
    predicted_binary = binarizer.transform(np.array(predictions).reshape(-1, 1)).flatten()

    precision = precision_score(actual_binary, predicted_binary, zero_division=1)
    recall = recall_score(actual_binary, predicted_binary, zero_division=1)
    f1 = f1_score(actual_binary, predicted_binary, zero_division=1)

    results.update({'precision': precision, 'recall': recall, 'f1': f1})

    if not config.ranking_metrics_flag or users is None:
        return results

    users = list(set(users))

    ndcg_scores = []
    average_precisions = []
    #reciprocal_ranks = []

    for user in users:
        user_predictions = [pred for idx, pred in enumerate(predictions) if users[idx] == user]
        user_references = [ref for idx, ref in enumerate(references) if users[idx] == user]

        user_df.loc[:, 'predicted'] = user_predictions
        user_df = user_df.sort_values(by='predicted', ascending=False)

        relevances = user_references

        ndcg = ndcg_at_k(relevances, config.k)
        ndcg_scores.append(ndcg)

        ap = average_precision(relevances, config.threshold_rating)
        average_precisions.append(ap)

        #for rank, relevance in enumerate(relevances, start=1):
        #    if relevance >= config.threshold_rating:
        #        reciprocal_ranks.append(1 / rank)
        #        break
        #else:
        #    reciprocal_ranks.append(0)

    ndcg = np.mean(ndcg_scores)
    map = np.mean(average_precisions)
    #mrr = np.mean(reciprocal_ranks)

    results.update({'ndcg': ndcg, 'map': map})

    return results


def review_evaluation(config, predictions: List[str], references: List[str]) -> Dict[str, Any]:
    references_list = [[ref] for ref in references]

    bleu_metric = evaluate.load("bleu")
    bleu_results = bleu_metric.compute(predictions=predictions, references=references_list)
    bleu_results["precision"] = np.mean(bleu_results["precisions"])

    bertscore_metric = evaluate.load("bertscore")
    bertscore_results = bertscore_metric.compute(
        predictions=predictions, references=references, lang=config.lang
    )
    bertscore_results["precision"] = np.mean(bertscore_results["precision"])
    bertscore_results["recall"] = np.mean(bertscore_results["recall"])
    bertscore_results["f1"] = np.mean(bertscore_results["f1"])

    #meteor_metric = evaluate.load("meteor")
    #meteor_results = meteor_metric.compute(predictions=predictions, references=references)

    rouge_metric = evaluate.load("rouge")
    rouge_results = rouge_metric.compute(predictions=predictions, references=references)

    return {
        "n_examples": len(predictions),
        #"meteor": float(meteor_results["meteor"]),
        "bleu": float(bleu_results["bleu"]),
        "rouge1": float(rouge_results["rouge1"]),
        "rouge2": float(rouge_results["rouge2"]),
        "rougeL": float(rouge_results["rougeL"]),
        "rougeLsum": float(rouge_results["rougeLsum"]),
        "bertscore.precision": float(bertscore_results["precision"]),
        "bertscore.recall": float(bertscore_results["recall"]),
        "bertscore.f1": float(bertscore_results["f1"]),
    }


def dcg_at_k(relevances, k):
    relevances = np.asarray(relevances)[:k]
    positions = np.arange(1, len(relevances) + 1)
    return np.sum(relevances / np.log2(positions + 1))


def dcg_at_k_pytorch(relevances, k, device):
    relevances = relevances[:k]
    positions = torch.arange(1, len(relevances) + 1, dtype=torch.float32).to(device)
    return torch.sum(relevances / torch.log2(positions + 1))


def idcg_at_k(relevances, k):
    sorted_relevances = sorted(relevances, reverse=True)
    return dcg_at_k(sorted_relevances, k)


def ndcg_at_k(relevances, k):
    dcg = dcg_at_k(relevances, k)
    idcg = idcg_at_k(relevances, k)
    return dcg / idcg if idcg > 0 else 0


def ndcg_at_k_pytorch(relevances, k, device):
    dcg = dcg_at_k_pytorch(relevances, k, device)
    ideal_relevances = torch.sort(relevances, descending=True).values
    idcg = dcg_at_k_pytorch(ideal_relevances, k, device)
    return dcg / idcg if idcg > 0 else torch.tensor(0.0).to(device)


def calculate_ndcg(df, aspect, predictions, config):
    k = config.k
    users = df['user_id'].unique()

    ndcg_scores = []

    for user in users:
        user_df = df[df['user_id'] == user]
        user_predictions = [pred for idx, pred in enumerate(predictions) if df.iloc[idx]['user_id'] == user]

        user_df.loc[:, 'predicted'] = user_predictions
        user_df = user_df.sort_values(by='predicted', ascending=False)

        relevances = user_df[aspect].values

        ndcg = ndcg_at_k(relevances, k)
        ndcg_scores.append(ndcg)

    mean_ndcg = np.mean(ndcg_scores)

    return mean_ndcg


def precision_at_k(relevances, k):
    relevances = np.asarray(relevances)[:k]
    return np.mean(relevances)


def average_precision(relevances, threshold_rating):
    relevances = np.asarray(relevances)
    precisions = []
    num_relevant = 0

    for k in range(1, len(relevances) + 1):
        if relevances[k - 1] >= threshold_rating:
            num_relevant += 1
            precisions.append(num_relevant / k)

    if num_relevant == 0:
        return 0

    return np.mean(precisions)


def average_precision_pytorch(relevances, threshold_rating, device):
    precisions = []
    num_relevant = 0
    for k in range(1, len(relevances) + 1):
        if relevances[k - 1] >= threshold_rating:
            num_relevant += 1
            precisions.append(num_relevant / k)

    if num_relevant == 0:
        return torch.tensor(0.0).to(device)

    return torch.tensor(precisions).mean().to(device)


def calculate_map(df, aspect, predictions, config):
    users = df['user_id'].unique()
    average_precisions = []

    for user in users:
        user_df = df[df['user_id'] == user].copy()
        user_predictions = [pred for idx, pred in enumerate(predictions) if df.iloc[idx]['user_id'] == user]

        user_df.loc[:, 'predicted'] = user_predictions
        user_df = user_df.sort_values(by='predicted', ascending=False)

        relevances = user_df[aspect].values

        ap = average_precision(relevances, config.threshold_rating)
        average_precisions.append(ap)

    mean_map = np.mean(average_precisions)

    return mean_map


def calculate_mrr(df, aspect, predictions, config):
    users = df['user_id'].unique()
    reciprocal_ranks = []

    for user in users:
        user_df = df[df['user_id'] == user].copy()
        user_predictions = [pred for idx, pred in enumerate(predictions) if df.iloc[idx]['user_id'] == user]

        user_df.loc[:, 'predicted'] = user_predictions
        user_df = user_df.sort_values(by='predicted', ascending=False)

        relevances = user_df[aspect].values

        for rank, relevance in enumerate(relevances, start=1):
            if relevance >= config.threshold_rating:
                reciprocal_ranks.append(1 / rank)
                break
        else:
            reciprocal_ranks.append(0)

    mean_mrr = np.mean(reciprocal_ranks)

    return mean_mrr


### Utils

In [7]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)


def empty_cache():
    with torch.no_grad():
        torch.cuda.empty_cache()


class Vocabulary:

    def __init__(self):
        self._elements2ids = {}
        self._ids2elements = {}
        self.n_elements = 0
        self.default_add = True

    def add_element(self, element: Union[int, float, str]):
        if element not in self._elements2ids:
            self._elements2ids[element] = self.n_elements
            self._ids2elements[self.n_elements] = element
            self.n_elements += 1

    def add_elements(self, elements: List[Union[int, float, str]]):
        for element in tqdm(elements, "Vocabulary creation", colour="green"):
            self.add_element(element)

    def __len__(self):
        return self.n_elements

    def id2element(self, id: int) -> Union[int, float, str]:
        return self._ids2elements[id]

    def element2id(self, element: Union[int, float, str]) -> int:
        if element not in self._elements2ids:
            if self.default_add:
                self.add_element(element)
            else:
                return None
        return self._elements2ids[element]

    def ids2elements(self, ids: List[int]) -> List[Union[int, float, str]]:
        return [self._ids2elements[id] for id in ids]

    def elements2ids(self, elements: List[Union[int, float, str]]) -> List[int]:
        return [self.element2id(element) for element in elements]

    def save(self, path: str):
        with open(path, "w") as f:
            json.dump({"elements2ids": self._elements2ids, "ids2elements": self._ids2elements, "n_elements": self.n_elements}, f)

    def load(self, path: str):
        with open(path, "r") as f:
            data = json.load(f)
            self._elements2ids = data["elements2ids"]
            self._ids2elements = data["ids2elements"]
            self.n_elements = data["n_elements"]


def create_vocab_from_df(metadata_df: pd.DataFrame, element_column: str) -> Vocabulary:
    elements = metadata_df[element_column].unique()
    vocab = Vocabulary()
    vocab.add_elements(elements)
    return vocab


def to_vocab_id(element, vocabulary: Vocabulary) -> int:
    return vocabulary.element2id(element)


def to_class_id(rating: float, n_classes: int, args: Any) -> int:
    scale = (args.max_rating - args.min_rating) / n_classes
    c = math.ceil((rating - args.min_rating) / scale) - 1
    if c < 0: c = 0
    if c >= n_classes: c = n_classes - 1
    return c


def save_model(model, save_model_path: str):
    torch.save(model.state_dict(), save_model_path)


def load_model(model, save_model_path: str):
    model.load_state_dict(torch.load(save_model_path))


def delete_punctuation(text: str) -> str:
    punctuation = r"[\!\"\#\$\%\&\'\(\)\*\+\,\-\.\/\:\;\<\=\>\?\@\[\\\]\^\_\`\{\|\}\~\n\t]"
    text = re.sub(punctuation, " ", text)
    text = re.sub('( )+', ' ', text)
    return text


def delete_stopwords(text: str) -> str:
    stop_words = set(nltk.corpus.stopwords.words('english'))
    return ' '.join([w for w in text.split() if w not in stop_words])


def delete_non_ascii(text: str) -> str:
    return ''.join([w for w in text if ord(w) < 128])


def replace_maj_word(text: str) -> str:
    token = '<MAJ>'
    return ' '.join([w if not w.isupper() else token for w in delete_punctuation(text).split()])


def delete_digit(text: str) -> str:
    return re.sub('[0-9]+', '', text)


def first_line(text: str) -> str:
    return re.split(r'[.!?]', text)[0]


def last_line(text: str) -> str:
    if text.endswith('\n'): text = text[:-2]
    return re.split(r'[.!?]', text)[-1]


def delete_balise(text: str) -> str:
    return re.sub("<.*?>", "", text)


def stem(text: str) -> str:
    stemmer = EnglishStemmer()
    tokens = nltk.word_tokenize(text)
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    stemmed_text = " ".join(stemmed_tokens)
    return stemmed_text


def lemmatize(text: str) -> str:
    lemmatizer = WordNetLemmatizer()
    tokens = nltk.word_tokenize(text)
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    lemmatized_text = " ".join(lemmatized_tokens)
    return lemmatized_text


def preprocess_text(text: str, args: Any, max_length: int=-1) -> str:
    text = str(text).strip()
    if args.replace_maj_word_flag: text = replace_maj_word(text)
    if args.lower_flag: text = text.lower()
    if args.delete_punctuation_flag: text = delete_punctuation(text)
    if args.delete_balise_flag: text = delete_balise(text)
    if args.delete_stopwords_flag: text = delete_stopwords(text)
    if args.delete_non_ascii_flag: text = delete_non_ascii(text)
    if args.delete_digit_flag: text = delete_digit(text)
    if args.first_line_flag: text = first_line(text)
    if args.last_line_flag: text = last_line(text)
    if args.stem_flag: text = stem(text)
    if args.lemmatize_flag: text = lemmatize(text)
    if max_length > 0 and args.truncate_flag:
        text = str(text).strip().split()
        if len(text) > max_length:
            text = text[:max_length - 1] + ["..."]
        text = " ".join(text)
    return text


### Training

In [8]:
def train(model: AURA, config, optimizer, dataloader, training_phase=0):
    model.train()

    losses = {"total": 0.0}
    if training_phase == 0:
        losses.update({"rating": 0.0, "overall_rating": 0.0, "aspect_rating": 0.0, "review_rating": 0.0})
    elif training_phase == 1:
        losses.update({"review": 0.0})

    for batch in tqdm(dataloader, f"Training {training_phase}", colour="cyan", total=len(dataloader)):
        U_ids = torch.LongTensor(batch["user_id"]).to(config.device) # (batch_size,)
        U_document_tokens = torch.LongTensor(batch["user_document_tokens"]).to(config.device) # (batch_size, n_sentences, sentence_length)
        U_masks = torch.LongTensor(batch["user_document_masks"]).to(config.device) # (batch_size, n_sentences, sentence_length)
        U_ratings = torch.tensor(batch["user_rating"], dtype=torch.float32).to(config.device) # (batch_size,)

        I_ids = torch.LongTensor(batch["item_id"]).to(config.device) # (batch_size,)
        I_document_tokens = torch.LongTensor(batch["item_document_tokens"]).to(config.device) # (batch_size, n_sentences, sentence_length)
        I_masks = torch.LongTensor(batch["item_document_masks"]).to(config.device) # (batch_size, n_sentences, sentence_length)
        I_ratings = torch.tensor(batch["item_rating"], dtype=torch.float32).to(config.device) # (batch_size,)

        if training_phase == 0:
            R = torch.tensor(batch["overall_rating"], dtype=torch.float32).to(config.device) # (batch_size,)
            A_ratings = torch.tensor(batch["aspects_ratings"], dtype=torch.float32).to(config.device) # (batch_size, n_aspects)
            UI_reviews_tokens = None

        elif training_phase == 1:
            R = None
            A_ratings = None
            UI_reviews_tokens = torch.LongTensor(batch["review_tokens"]).to(config.device) # (batch_size, review_length)

        output = model(
            U_ids, I_ids,
            U_document_tokens, U_masks, U_ratings,
            I_document_tokens, I_masks, I_ratings,
            R, A_ratings,
            UI_reviews_tokens,
            inference_flag=False
        )

        loss = output["losses"]["total"]

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        for loss in output["losses"]:
            losses[loss] += output["losses"][loss].item()

    for loss in losses:
        losses[loss] /= len(dataloader)
    return losses


def eval(model: AURA, config, dataloader):
    model.eval()

    users = []
    references = {}
    predictions = {}
    for key in ["overall_rating", "aspect_rating", "user_rating", "item_rating", "review",
               *([f"{aspect}_rating" for aspect in config.aspects])]:
        references[key] = []
        predictions[key] = []

    for batch_idx, batch in tqdm(enumerate(dataloader), "Evaluation", colour="cyan", total=len(dataloader)):
        U_ids = torch.LongTensor(batch["user_id"]).to(config.device) # (batch_size,)
        U_document_tokens = torch.LongTensor(batch["user_document_tokens"]).to(config.device) # (batch_size, n_sentences, sentence_length)
        U_masks = torch.LongTensor(batch["user_document_masks"]).to(config.device) # (batch_size, n_sentences, sentence_length)
        U_ratings = torch.tensor(batch["user_rating"], dtype=torch.float32).to(config.device) # (batch_size,)

        I_ids = torch.LongTensor(batch["item_id"]).to(config.device) # (batch_size,)
        I_document_tokens = torch.LongTensor(batch["item_document_tokens"]).to(config.device) # (batch_size, n_sentences, sentence_length)
        I_masks = torch.LongTensor(batch["item_document_masks"]).to(config.device) # (batch_size, n_sentences, sentence_length)
        I_ratings = torch.tensor(batch["item_rating"], dtype=torch.float32).to(config.device) # (batch_size,)

        R = torch.tensor(batch["overall_rating"], dtype=torch.float32).to(config.device) # (batch_size,)
        A_ratings = torch.tensor(batch["aspects_ratings"], dtype=torch.float32).to(config.device) # (batch_size, n_aspects)
        UI_reviews_tokens = torch.LongTensor(batch["review_tokens"]).to(config.device) # (batch_size, sentence_length)
        reviews = (batch["review"]) # (batch_size,)

        output = model(
            U_ids, I_ids,
            U_document_tokens, U_masks, U_ratings,
            I_document_tokens, I_masks, I_ratings,
            R, A_ratings,
            UI_reviews_tokens,
            inference_flag=True
        )

        R_hat = output["overall_rating"]
        A_ratings_hat = output["aspects_ratings"]
        reviews_hat = output["review"]

        U_ratings_hat = output["user"]["document"]["rating"]
        I_ratings_hat = output["item"]["document"]["rating"]

        users.extend(U_ids.cpu().detach().tolist())

        references["overall_rating"].extend(R.cpu().detach().tolist())
        predictions["overall_rating"].extend(R_hat.cpu().detach().tolist())

        references["review"].extend(reviews)
        predictions["review"].extend(reviews_hat)

        references["user_rating"].extend(U_ratings.cpu().detach().tolist())
        predictions["user_rating"].extend(U_ratings_hat.cpu().detach().tolist())
        references["item_rating"].extend(I_ratings.cpu().detach().tolist())
        predictions["item_rating"].extend(I_ratings_hat.cpu().detach().tolist())

        for a, aspect in enumerate(config.aspects):
            references[f"{aspect}_rating"].extend(A_ratings[:, a].cpu().detach().tolist())
            predictions[f"{aspect}_rating"].extend(A_ratings_hat[:, a].cpu().detach().tolist())

        if config.verbose and batch_idx == 0:
            for i in range(len(reviews)):
                log = "\n" + "\n".join([
                    f"User ID: {U_ids[i]}",
                    f"Item ID: {I_ids[i]}",
                    f"Overall Rating: Actual={R[i]:.4f} Predicted={R_hat[i]:4f}",
                    *[f"{aspect} Rating: Actual={A_ratings[i][a]:.4f} Predicted={A_ratings_hat[i][a]:.4f}"
                    for a, aspect in enumerate(config.aspects)],
                    "Review:",
                    f"\tGround Truth: {reviews[i]}",
                    f"\tGenerated: {reviews_hat[i]}",
                    f"-" * 80
                ])
                print(log)
                with open(config.log_file_path, "w", encoding="utf-8") as log_file:
                    log_file.write(log)

    scores = {}
    for metric in references:
        if metric == "review":
            scores[metric] = review_evaluation(config, predictions[metric], references[metric])
        elif metric in ["user_rating", "item_rating"]:
            scores[metric] = rating_evaluation_pytorch(config, predictions[metric], references[metric])
        else:
            scores[metric] = rating_evaluation_pytorch(config, predictions[metric], references[metric], users)
    return scores


def trainer(model: AURA, config, train_dataloader, eval_dataloader):
    optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)

    train_infos = {}
    eval_infos = {}

    best_rating = float("inf")
    best_review = -float("inf")

    config.n_epochs = config.n_ratings_epochs + config.n_reviews_epochs
    training_phase = 0
    model.set_training_phase(training_phase)

    progress_bar = tqdm(range(1, 1 + config.n_epochs), "Training", colour="blue")
    for epoch in progress_bar:
        empty_cache()

        if epoch > config.n_ratings_epochs:
            training_phase = 1
            model.set_training_phase(training_phase)

        losses = train(model, config, optimizer=optimizer, dataloader=train_dataloader, training_phase=training_phase)

        for loss in losses.keys():
            if loss not in train_infos.keys():
                train_infos[loss] = []
            train_infos[loss].append(losses[loss])

        train_loss = losses["total"]
        desc = (
            f"[{epoch} / {config.n_epochs}] Loss: {train_loss:.4f} " +
            f"Best: {config.rating_metric}={best_rating:.4f} {config.review_metric}={best_review:.4f}"
        )

        if epoch % config.eval_every == 0:
            with torch.no_grad():
                scores = eval(model, config, dataloader=eval_dataloader)

            for metric_set in scores.keys():
                if metric_set not in eval_infos.keys():
                    eval_infos[metric_set] = {}
                for metric in scores[metric_set].keys():
                    if metric not in eval_infos[metric_set].keys():
                        eval_infos[metric_set][metric] = []
                    eval_infos[metric_set][metric].append(scores[metric_set][metric])

            eval_rating = scores["overall_rating"][config.rating_metric]
            eval_review = scores["review"][config.review_metric]

            if training_phase == 0:
                if eval_rating < best_rating:
                    save_model(model, config.save_model_path)
                    best_rating = eval_rating

            elif training_phase == 1:
                if eval_review > best_review:
                    save_model(model, config.save_model_path)
                    best_review = eval_review

            desc = (
                f"[{epoch} / {config.n_epochs}] " +
                f"Loss: train={train_loss:.4f} " +
                f"Test: {config.rating_metric}={eval_rating:.4f} {config.review_metric}={eval_review:.4f} " +
                f"Best: {config.rating_metric}={best_rating:.4f} {config.review_metric}={best_review:.4f}"
            )

        progress_bar.set_description(desc)

        results = {"train": train_infos, "eval": eval_infos}
        with open(config.res_file_path, "w") as res_file:
            json.dump(results, res_file)

    return train_infos, eval_infos

### Experiments : Hotels (TripAdvisor)

In [9]:
class Config:
    pass

config = Config()

config.base_dir = os.path.join(".")
config.dataset_dir = os.path.join("Hotels")
config.dataset_name = "Hotels"
config.dataset_path = os.path.join( "Hotels", "data.csv")
config.aspects = ["service", "cleanliness", "value", "sleep_quality", "rooms", "location"]
config.n_aspects = len(config.aspects)
config.aspects_sep = " "
config.lang = "en"
config.min_rating = 1.0
config.max_rating = 5.0

config.exp_name = "test"
config.review_flag = True
config.rating_flag = True
config.model_name_or_path = "t5-small"
config.d_words = 512
config.d_model = 128
config.n_sentences = 16
config.sentence_length = 32
config.review_length = 512

config.n_prompt = 32
config.prompt_tuning_flag = True
config.alpha = 1
config.beta = 1
config.gamma = 1
config.lambda_ = 1

config.save_model_path = ""
config.n_ratings_epochs = 16
config.n_reviews_epochs = 16
config.lr = 1e-3
config.batch_size = 32
config.train_size = .8
config.eval_size = .1
config.test_size = .1

config.seed = 42
config.rating_metric = "rmse"
config.review_metric = "bleu"
config.ranking_metrics_flag = False
config.threshold_rating = 4.0
config.k = 10
config.verbose = True
config.verbose_every = 1
config.eval_every = 8

config.truncate_flag = True
config.lower_flag = True
config.delete_balise_flag = True
config.delete_stopwords_flag = False
config.delete_punctuation_flag = False
config.delete_non_ascii_flag = True
config.delete_digit_flag = False
config.replace_maj_word_flag = False
config.first_line_flag = False
config.last_line_flag = False
config.stem_flag = False
config.lemmatize_flag = False

In [10]:
set_seed(config.seed)

if config.dataset_dir == "":
    config.dataset_dir = os.path.join(config.base_dir, config.dataset_name)
if config.dataset_path == "":
    config.dataset_path = os.path.join(config.dataset_dir, "data.csv")

data_df = pd.read_csv(config.dataset_path)
(train_df, eval_df, test_df), (users_vocab, items_vocab) = process_data(data_df, config)
#train_df = train_df.head(100)
#eval_df = eval_df.head(100)
#test_df = test_df.head(100)

Vocabulary creation:   0%|          | 0/8830 [00:00<?, ?it/s]

Vocabulary creation:   0%|          | 0/2903 [00:00<?, ?it/s]

In [11]:
config.n_users = len(users_vocab)
config.n_items = len(items_vocab)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
config.device = device

exps_base_dir = os.path.join(config.dataset_dir, "exps")
exp_dir = os.path.join(exps_base_dir, config.exp_name)
os.makedirs(exp_dir, exist_ok=True)
config.exp_dir = exp_dir
config.log_file_path = os.path.join(exp_dir, "log.txt")
config.res_file_path = os.path.join(exp_dir, "res.json")

if config.save_model_path == "":
    config.save_model_path = os.path.join(exp_dir, "model.pth")

In [12]:
tokenizer = T5Tokenizer.from_pretrained(config.model_name_or_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [13]:
train_dataset = RatingsReviewDataset(train_df, config, tokenizer)
eval_dataset = RatingsReviewDataset(eval_df, config, tokenizer)
test_dataset = RatingsReviewDataset(test_df, config, tokenizer)

Processing data:   0%|          | 0/64705 [00:00<?, ?it/s]

Processing data:   0%|          | 0/8088 [00:00<?, ?it/s]

Processing data:   0%|          | 0/8088 [00:00<?, ?it/s]

In [14]:
train_dataloader = DataLoader(
    train_dataset, batch_size=config.batch_size, shuffle=True, collate_fn=collate_fn,
    num_workers=4, pin_memory=True
)
eval_dataloader = DataLoader(
    eval_dataset, batch_size=config.batch_size, shuffle=False, collate_fn=collate_fn,
    num_workers=4, pin_memory=True
)
test_dataloader = DataLoader(
    test_dataset, batch_size=config.batch_size, shuffle=False, collate_fn=collate_fn,
    num_workers=4, pin_memory=True
)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [15]:
t5_model = T5ForConditionalGeneration.from_pretrained(config.model_name_or_path)
if config.prompt_tuning_flag:
    for param in t5_model.parameters():
        param.requires_grad = False
text_module = T5TextModule(config, t5_model)
text_module.to(config.device)
print()

In [16]:
if config.verbose:
    log = "\n" + (
        f"Model name: {config.model_name_or_path}\n" +
        f"Dataset: {config.dataset_name}\n" +
        f"Aspects: {config.aspects}\n" +
        f"#Users: {config.n_users}\n" +
        f"#Items: {config.n_items}\n" +
        f"Device: {device}\n\n" +
        f"Args:\n{config}\n\n" +
        #f"Model: {model}\n\n" +
        f"Data:\n{train_df.head(5)}\n\n"
    )
    print(log)
    with open(config.log_file_path, "w", encoding="utf-8") as log_file:
        log_file.write(log)


Model name: t5-small
Dataset: Hotels
Aspects: ['service', 'cleanliness', 'value', 'sleep_quality', 'rooms', 'location']
#Users: 8830
#Items: 2903
Device: cuda

Args:

Data:
       user_id  item_id  rating  \
28086       15      556     5.0   
75222     4520       60     3.0   
58839      298      373     4.0   
49560     3953     1260     3.0   
59308       15      392     1.0   

                                                  review  \
28086  The reception staff were very helpful. The loc...   
75222  We have stayed here previously when the hotel ...   
58839  The hotel was nice and just a 2 block walk to ...   
49560  My husband and I stayed here for a few nights ...   
59308  We ended up cancelling our booking because we ...   

                        review_title     timestamp  service  cleanliness  \
28086             “IBIS Monte Marte”  1.322006e+09      5.0          5.0   
75222  “It was better as a Marriott”  1.325981e+09      3.0          3.0   
58839                “Good

In [17]:
model = AURA(config, text_module=text_module, tokenizer=tokenizer)
model.to(config.device)
#if config.save_model_path != "":
    #load_model(model, config.save_model_path)
print()

In [ ]:
train_infos, eval_infos = trainer(model, config, train_dataloader, eval_dataloader)

Training:   0%|          | 0/32 [00:00<?, ?it/s]

Training 0:   0%|          | 0/2023 [00:00<?, ?it/s]

Training 0:   0%|          | 0/2023 [00:00<?, ?it/s]

Training 0:   0%|          | 0/2023 [00:00<?, ?it/s]

In [ ]:
load_model(model, config.save_model_path)
with torch.no_grad():
    test_infos = eval(model, config, dataloader=test_dataloader)

In [ ]:
results = {"test": test_infos, "train": train_infos, "eval": eval_infos}
with open(config.res_file_path, "w") as res_file:
    json.dump(results, res_file)
print(results)